In [4]:
import pandas as pd


import numpy as np
from sklearn import preprocessing


from lib.preprop import *
from lib.geo_to_vector import vectorize_geo
from lib.eda_visualization import *

Loading the dataframe

In [2]:
df = pd.read_csv('df_complete.csv').iloc[:,3:]
df.shape

/tmp/ipykernel_45404/1107331204.py:1: DtypeWarning: Columns (18,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_complete.csv').iloc[:,3:]


(13048, 2868)

Removing duplicated rows

In [3]:
df_no_dups = remove_duplicates(df,['company_name'])
df_no_dups.shape

(10070, 2868)

Sorting all the columns by groups

In [6]:
cat_cols = ['company_name','company_about', 'founded', 'business model','employees','product stage','status','geographical markets','fund_stage',]
num_cols = ['total_raised','total_rounds', 'investors','ipo_price']
tag_cols = [col for col in df_no_dups.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in df_no_dups.columns if col.startswith('targetmarket_')]
sector_list = [col for col in df_no_dups.columns if col.startswith("sector_")]
target_ind_list = [col  for col in df_no_dups.columns if col.startswith("target_industry_")]
technology_list = [col  for col in df_no_dups.columns if col.startswith("core_technology_")]


NameError: name 'df_no_dups' is not defined

<h1>Repairing columns</h1>
<p> Before removing NaN values, we will first identify what values we expect to have in these columns and we will implement it</p>

<h2> Column 'founded'  </h2>
<p>We will change the string representation of founded - "month/year" to an integer "year"</br>
Using REGEX to find the year</p>

In [5]:
# Fixing 'founded' column

print(f'type of "founded" col is : {df_no_dups.founded.dtype}')
print('\nRunning script..')
import re
founded_pattern = r"(\d{4})"
year_list=[]
df_founded = df_no_dups.copy()

for i, val in enumerate(df_founded.founded) :
    year = val.split('/')[-1]
    m = re.search(founded_pattern, year)
    if m:
        year_list.append(int(m.group(0)))
        
        
df_founded['founded'] = year_list


print('\nOperation succesfull!')
print(f'type of "founded" col is : {df_founded.founded.dtype}')

df_founded.shape
        

type of "founded" col is : object

Running script..

Operation succesfull!
type of "founded" col is : int64


(10070, 2868)

<h2> Column 'geographical markets'</h2>
<p>The column contains a string representing the geographical markets the company aims for</br>
The script lib/geo_to_vector.py calculates the precentage of the market and add it to the dataframe. </p>

In [6]:
# Fixing 'geographical markets' column

print("Geographical markets col:")
print(df_founded['geographical markets'].head())
print('----------------------------------------')

print('\nRunning the script...\n')
df_geo_market = vectorize_geo(df_founded, 'l')

print('\n----------------------------------------')
print('Operation succesfull!\n')
print("Geographical percentage col:")
print(df_geo_market['geo_market_per'].head())

num_cols.append('geo_market_per')
cat_cols.remove('geographical markets')
df_geo_market = df_geo_market.drop(['geographical markets'], axis=1)
df_geo_market = df_geo_market.dropna(subset=['company_name'])

Geographical markets col:
0    australia, canada, france, india, united kingd...
1                                                  NaN
2                 canada, mexico, spain, united states
3                                global, united states
4    north america, europe, global, france, germany...
Name: geographical markets, dtype: object
----------------------------------------

Running the script...

shape of df['geo_market_per']: (10070,)

----------------------------------------
Operation succesfull!

Geographical percentage col:
0    0.239983
1         NaN
2    0.068498
3    1.000000
4    1.000000
Name: geo_market_per, dtype: float64


<h1> # Checkpoint : save the new df # </h1>

In [90]:
# df_geo_market.to_csv('cp1.csv')
df_geo_market = pd.read_csv('cp1.csv').iloc[:,1:]
df_geo_market

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (15,85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,company_name,company_about,founded,business model,employees,funding stage,raised,product stage,status,products,...,targetmarket_public-transportation,targetmarket_smart-mobility,tag_hydraulic-drive,tag_hud,tag_simulation-software,tag_luggage,tag_traffic-violations,tag_car-audio,tag_trip,geo_market_per
0,Tastewise,Tastewise is an AI platform designed to help f...,2017,B2B,51-200,ROUND A,$21.5M,Released,active,tastewise,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.239983
1,Wilk Technologies,Wilk is dedicated to revolutionizing the dairy...,2018,"B2B, B2B2C",11-50,Public,$4.69M,R&D,active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eco Pack Green Box,Eco Pack Green Box has developed and patented ...,2008,B2B,11-50,Revenue Financed,NaN,Released,not_active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068498
3,BeeHero,BeeHero has developed a platform that can pred...,2017,B2B,1-10,ROUND A,$24M,Released,active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
4,Cham Foods,Cham Foods is a multinational company with man...,1970,"B2B, B2B2C",11-50,Public,NaN,Released,active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10065,Intelligent Smart Ideas,Intelligent Smart Ideas is the creator of iSaf...,2017,"B2B, B2G",1-10,Bootstrapped,NaN,R&D,active,"isafecross, ibabysafe+, ibabycrytranslator",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10066,MappMakers,MappMakers has an algorithm that can take the ...,2016,B2C,1-10,NaN,NaN,NaN,not_active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10067,LYNX Smartcars,LYNX is developing software for connected and ...,2016,B2B,1-10,Bootstrapped,NaN,R&D,not_active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10068,Deeyook Location Technologies,Deeyook seeks to redefine location technology ...,2017,B2B,11-50,Seed,NaN,Released,active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


In [91]:
cat_cols = ['company_name','company_about', 'founded', 'business model','employees','product stage','status','fund_stage']
num_cols = ['total_raised','total_rounds', 'investors','ipo_price','geo_market_per']
tag_cols = [col for col in df_geo_market.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in df_geo_market.columns if col.startswith('targetmarket_')]
sector_list = [col for col in df_geo_market.columns if col.startswith("sector_")]
target_ind_list = [col  for col in df_geo_market.columns if col.startswith("target_industry_")]
technology_list = [col  for col in df_geo_market.columns if col.startswith("core_technology_")]



<h1>Handling NaN values</h1>

<p>First, we will check how many null values are in each column of the non-binary columns</p>

In [92]:
print("missing values:")
cols_to_check = num_cols + cat_cols
for col in cols_to_check:
    s = df_geo_market[col].isnull().sum()
    print(f'\t{col}: {s} missing values') 

missing values:
	total_raised: 5704 missing values
	total_rounds: 5704 missing values
	investors: 5704 missing values
	ipo_price: 9920 missing values
	geo_market_per: 2912 missing values
	company_name: 0 missing values
	company_about: 2 missing values
	founded: 0 missing values
	business model: 68 missing values
	employees: 32 missing values
	product stage: 163 missing values
	status: 0 missing values
	fund_stage: 5769 missing values


In [93]:
print(f"num cols: {num_cols}")
print(f"cat cols: {cat_cols}")

num cols: ['total_raised', 'total_rounds', 'investors', 'ipo_price', 'geo_market_per']
cat cols: ['company_name', 'company_about', 'founded', 'business model', 'employees', 'product stage', 'status', 'fund_stage']


In [94]:
print(f'before :{df_geo_market.shape}')

drop_cols = ['funding stage','products','raised']
new_df = df_geo_market.drop(drop_cols, axis =1)

new_df = remove_missing_str_val_rows(new_df, cat_cols)
new_df = repair_categorical_missing_vals(new_df, cat_cols)

new_df = repair_numeric_missing_vals_zero(new_df, [col for col in num_cols if col != 'geo_market_per'])
new_df = repair_numeric_missing_vals_median(new_df, ['geo_market_per'])

print(f'after :{new_df.shape}')

before :(10070, 2868)
after :(10070, 2865)


In [95]:
for col in num_cols:
    new_df[col] = conv_to_float(new_df[col])

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


<h3> Dealing with NaN in binary columns  </h3>

In [97]:
new_df[tag_cols] = new_df[tag_cols].fillna(0)
new_df[targetmarket_cols] = new_df[targetmarket_cols].fillna(0)
new_df[sector_list] = new_df[sector_list].fillna(0)
new_df[target_ind_list] = new_df[target_ind_list].fillna(0)
new_df[technology_list] = new_df[technology_list].fillna(0)

In [98]:
print("Checking missing values in non binary columns...\n")

for col in num_cols + cat_cols:
    s = new_df[col].isnull().sum()
    print(f'{col}: {s} missing values') 

Checking missing values in non binary columns...

total_raised: 0 missing values
total_rounds: 0 missing values
investors: 0 missing values
ipo_price: 0 missing values
geo_market_per: 0 missing values
company_name: 0 missing values
company_about: 0 missing values
founded: 0 missing values
business model: 0 missing values
employees: 0 missing values
product stage: 0 missing values
status: 0 missing values
fund_stage: 0 missing values


In [99]:
bin_cols = tag_cols + targetmarket_cols + sector_list + target_ind_list + technology_list
print("Checking missing values in binary columns...\n")
are_missing = False
for col in bin_cols:
    s = new_df[col].isnull().sum()
    if s!=0:
        are_missing = True
        print(f'{col}: {s} missing values') 

if are_missing is False:
    print("No Nan values in binary columns!")

Checking missing values in binary columns...

No Nan values in binary columns!


<h1> # Checkpoint 2 : save the new df # </h1>


In [100]:
new_df.to_csv('cp2.csv')
# new_df = pd.read_csv('cp2.csv').iloc[:,1:]
# new_df.shape

In [101]:
# Defining cols

cat_cols = ['company_name','company_about', 'founded', 'business model','employees','product stage','status','fund_stage']
num_cols = ['total_raised','total_rounds', 'investors','ipo_price', 'geo_market_per']
tag_cols = [col for col in new_df.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in new_df.columns if col.startswith('targetmarket_')]
sector_list = [col for col in new_df.columns if col.startswith("sector_")]
target_ind_list = [col  for col in new_df.columns if col.startswith("target_industry_")]
technology_list = [col  for col in new_df.columns if col.startswith("core_technology_")]
bin_cols = tag_cols + targetmarket_cols + sector_list + target_ind_list + technology_list

<h1> Categorical columns </h1>

<p> First, we need to explore the categorical columns:</p>

In [103]:
cat_df = new_df.copy()
cat_df[cat_cols].describe(include='all')

,company_name,company_about,founded,business model,employees,product stage,status,fund_stage
count,10070,10070,10070.000000,10070,10070,10070,10070,10070
unique,10070,10015,NaN,16,6,7,2,11
top,Tastewise,This company is a known business entity but la...,NaN,B2B,1-10,Released,active,na
freq,1,48,NaN,4743,6232,6674,6554,5769
mean,NaN,NaN,2011.537736,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,10.052327,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1901.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2010.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2014.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2017.000000,NaN,NaN,NaN,NaN,NaN


<p>We see that [employees, business model, status, product stage and fund stage] columns</br>
Are with few unique values, therefore we can encode them.</p>

In [104]:
# Employees col
print("Value counts for 'employees' column:\n")
print(cat_df.employees.value_counts())

print("\nApplying LabelEncoder.\n")

le = preprocessing.LabelEncoder()
cat_df.employees = le.fit_transform(cat_df.employees)
print("New value counts for 'employees' column:\n")
print(cat_df.employees.value_counts())

Value counts for 'employees' column:

1-10       6232
11-50      2683
51-200      822
201-500     189
500+        112
na           32
Name: employees, dtype: int64

Applying LabelEncoder.

New value counts for 'employees' column:

0    6232
1    2683
4     822
2     189
3     112
5      32
Name: employees, dtype: int64


We will do the same for the rest of the columns

In [105]:
cols = ['business model', 'product stage', 'fund_stage']

for col in cols:
    le = preprocessing.LabelEncoder()
    cat_df[col] = le.fit_transform(cat_df[col])
    print(f'{col} : {le.classes_}')

replace_map = {'active' : 1, 'not_active' : 0}
cat_df.status.replace(replace_map, inplace= True)

business model : ['B2B' 'B2B, B2B2C' 'B2B, B2C' 'B2B, B2C, B2B2C' 'B2B, B2C, B2G'
 'B2B, B2C, B2G, B2B2C' 'B2B, B2G' 'B2B, B2G, B2B2C' 'B2B2C' 'B2C'
 'B2C, B2B2C' 'B2C, B2G' 'B2C, B2G, B2B2C' 'B2G' 'B2G, B2B2C' 'na']
product stage : ['Alpha' 'Beta' 'Clinical Trial' 'Customer development' 'R&D' 'Released'
 'na']
fund_stage : ['A' 'Acquired' 'B' 'Bootstrapped' 'C+' 'Established' 'Pre-Seed' 'Public'
 'Revenue Financed' 'Seed' 'na']


In [74]:
def remove_punctuation(text):
    '''a function for removing punctuation'''
    text = ' '.join(text.split('-'))
    return text

In [75]:
word_df = pd.DataFrame(columns = ['value','instances','tag','targetmarket', 'sector', 'target_ind', 'core_technology'])

for i,col in enumerate(bin_cols):
    splitted = col.split('_')
    name = splitted[-1]
    for type in word_df.columns[2:]:
        word_df.loc[i, type] = col.startswith(type)
    word_df.loc[i,'value'] = remove_punctuation(name)
    word_df.loc[i,'instances'] = cat_df[col].sum()


word_df


,value,instances,tag,targetmarket,sector,target_ind,core_technology
0,computer vision,309.0,True,False,False,False,False
1,behavior analytics,351.0,True,False,False,False,False
2,machine learning,1152.0,True,False,False,False,False
3,consumer packaged goods,8.0,True,False,False,False,False
4,natural language processing,294.0,True,False,False,False,False
...,...,...,...,...,...,...,...
2845,Nonwoven Fabric,5.0,False,False,False,False,True
2846,Insulation,5.0,False,False,False,False,True
2847,Scent,4.0,False,False,False,False,True
2848,Exoskeleton,1.0,False,False,False,False,True


In [106]:
cat_df

,company_name,company_about,founded,business model,employees,product stage,status,fund_stage,total_raised,total_rounds,...,targetmarket_public-transportation,targetmarket_smart-mobility,tag_hydraulic-drive,tag_hud,tag_simulation-software,tag_luggage,tag_traffic-violations,tag_car-audio,tag_trip,geo_market_per
0,Tastewise,Tastewise is an AI platform designed to help f...,2017,0,4,5,1,0,21500000,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.239983
1,Wilk Technologies,Wilk is dedicated to revolutionizing the dairy...,2018,1,1,4,1,7,4690000,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.866298
2,Eco Pack Green Box,Eco Pack Green Box has developed and patented ...,2008,0,1,5,0,10,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068498
3,BeeHero,BeeHero has developed a platform that can pred...,2017,0,0,5,1,0,24000000,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
4,Cham Foods,Cham Foods is a multinational company with man...,1970,1,1,5,1,10,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10065,Intelligent Smart Ideas,Intelligent Smart Ideas is the creator of iSaf...,2017,6,0,4,1,10,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.866298
10066,MappMakers,MappMakers has an algorithm that can take the ...,2016,9,0,6,0,10,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.866298
10067,LYNX Smartcars,LYNX is developing software for connected and ...,2016,0,0,4,0,10,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.866298
10068,Deeyook Location Technologies,Deeyook seeks to redefine location technology ...,2017,0,1,5,1,10,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000


<h1>Final cleaned dataframe</h1>
<p>Saving the df as csv file in the following col order</br>
cat_cols, num_cols, tag_cols, targetmarket_cols, sector_list, target_ind_list, technology_list</p>

In [107]:
# Final cleaned non-binary cols

final_df = pd.DataFrame(cat_df, columns = cat_cols + num_cols + tag_cols + targetmarket_cols + sector_list + target_ind_list + technology_list)
final_df.to_csv('final_cleaned.csv')